In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.schema.runnable import RunnablePassthrough

chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    tiktoken_model_name ="gpt-3.5-turbo",
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)
memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_token_limit=120,
    return_messages=True
)

examples = [
    {
        "movie":"The God father I",
        "answer": """
        The God father I -> 🤵🏻‍♂️🔫🕴️
        """ 
    },
    {
        "movie":"Blade Runner",
        "answer": """
        Blade Runner -> 🌆👁️‍🗨️💭
        """
    },
    {
        "movie":"Braveheart",
        "answer": """
        Braveheart -> ⚔️🏴󠁧󠁢󠁳󠁣󠁴󠁿🔥
        """
    },
]

example_prompt = ChatPromptTemplate.from_messages([  #prompt의 예시를 만들어 주고
    ("human","Describe this {movie} in three emojis."),
    ("ai","{answer}")
])

few_chat_prompt = FewShotChatMessagePromptTemplate(  #few_chat_prompt를 만들기 위해서 example_prompt와 examples를 만들어서 넣었다.
    example_prompt = example_prompt, #Prompt의 예시와 
    examples = examples, #example을 집어 넣었고
)

final_prompt = ChatPromptTemplate.from_messages([ #최종 prompt를 만들기위해서 few_chat_prompt를 가지고 왔고, 기존의 데이터를 저장하기 위해 MessagePlaceholder를 만들어서 넣었다.
    ("system","You're a film critic and expert. Please describe the film I'm suggesting using three emojis."), 
    few_chat_prompt,
    MessagesPlaceholder(variable_name="history"),
    ("human", "{movie}")
    ])

def load_memory(_): #load_memory는 인자를 받아야 함.
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory)| final_prompt | chat #assign에 넣어줄때는 변수에 변수를 넣는거라. dic구조 필요없음.

def invoke_chain(movie):
    result = chain.invoke(
        {
            "movie": movie,
        }
    )
    memory.save_context({"input":movie},{"output": result.content}) #input에 movie를 써야함.
    
invoke_chain("Avata")



        Avatar -> 🌌🌿👽

In [6]:
invoke_chain("The God father")


        The Godfather -> 🤵🏻🔫💼

In [7]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Avata'),
  AIMessage(content='\n        Avatar -> 🌌🌿👽'),
  HumanMessage(content='The God father'),
  AIMessage(content='\n        The Godfather -> 🤵🏻🔫💼')]}

In [8]:
invoke_chain("What movie did you enter before?")

I previously described "The Godfather" using the emojis 🤵🏻🔫💼. If you need a different description or have another movie in mind, feel free to ask!